<a href="https://colab.research.google.com/github/armandordorica/velocity_limits/blob/main/Velocity_limits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/armandordorica/velocity_limits.git

Cloning into 'velocity_limits'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 38 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [2]:
cd velocity_limits/takehome/

/content/velocity_limits/takehome


In finance, it's common for accounts to have so-called "velocity limits". In this task, you'll write a program that accepts or declines attempts to load funds into customers' accounts in real-time.

Each attempt to load funds will come as a single-line JSON payload, structured as follows:

```json
{ "id": "1234", "customer_id": "1234", "load_amount": "$123.45", "time": "2018-01-01T00:00:00Z" }
```

Each customer is subject to three limits:

- A maximum of $\$5,000$ can be loaded per day
- A maximum of $\$20,000$ can be loaded per week
- A maximum of 3 loads can be performed per day, regardless of amount

As such, a user attempting to load $\$3,000$ twice in one day would be declined on the second attempt, as would a user attempting to load $400 four times in a day.

For each load attempt, you should return a JSON response indicating whether the fund load was accepted based on the user's activity, with the structure:

```json
{ "id": "1234", "customer_id": "1234", "accepted": true }
```

You can assume that the input arrives in ascending chronological order and that if a load ID is observed more than once for a particular user, all but the first instance can be ignored. Each day is considered to end at midnight UTC, and weeks start on Monday (i.e. one second after 23:59:59 on Sunday).

Your program should process lines from `input.txt` and return output in the format specified above, either to standard output or a file. Expected output given our input data can be found in `output.txt`.

You're welcome to write your program in a general-purpose language of your choosing.

We value well-structured, self-documenting code with sensible test coverage. Descriptive function and variable names are appreciated, as is isolating your business logic from the rest of your code.


### Importing Source data

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

input_data = pd.read_json('input.txt', lines=True)
input_data.head(), len(input_data)

(      id  customer_id load_amount                  time
 0  15887          528    $3318.47  2000-01-01T00:00:00Z
 1  30081          154    $1413.18  2000-01-01T01:01:22Z
 2  26540          426     $404.56  2000-01-01T02:02:44Z
 3  10694            1     $785.11  2000-01-01T03:04:06Z
 4  15089          205    $2247.28  2000-01-01T04:05:28Z, 1000)

In [4]:
input_data.head()

,id,customer_id,load_amount,time
0,15887,528,$3318.47,2000-01-01T00:00:00Z
1,30081,154,$1413.18,2000-01-01T01:01:22Z
2,26540,426,$404.56,2000-01-01T02:02:44Z
3,10694,1,$785.11,2000-01-01T03:04:06Z
4,15089,205,$2247.28,2000-01-01T04:05:28Z


In [5]:
len(input_data['id']), len(input_data['id'].unique())

(1000, 984)

**"If a load ID is observed more than once for a particular user, all but the first instance can be ignored."** 
* There are 984 unique `id`s

Looking at unique combinations of load `id` and `customer_id`

In [6]:
input_data['unique_id']  = input_data['id'].astype(str) + "_" + input_data['customer_id'].astype(str)

In [7]:
input_data.head()

,id,customer_id,load_amount,time,unique_id
0,15887,528,$3318.47,2000-01-01T00:00:00Z,15887_528
1,30081,154,$1413.18,2000-01-01T01:01:22Z,30081_154
2,26540,426,$404.56,2000-01-01T02:02:44Z,26540_426
3,10694,1,$785.11,2000-01-01T03:04:06Z,10694_1
4,15089,205,$2247.28,2000-01-01T04:05:28Z,15089_205


### 1. Keeping only the first instance for any combination of load id and customer id

In [8]:
grouped_input_data = input_data.groupby(["unique_id"])
input_data = grouped_input_data.first().copy(deep=True)

In [9]:
input_data.head()

,id,customer_id,load_amount,time
unique_id,,,,
10002_35,10002,35,$2385.52,2000-01-20T05:16:22Z
10041_239,10041,239,$5455.00,2000-01-19T00:38:06Z
10041_596,10041,596,$162.67,2000-02-04T22:40:40Z
10047_137,10047,137,$3266.77,2000-01-19T13:55:52Z
10055_460,10055,460,$2671.07,2000-02-08T21:47:46Z


In [10]:
input_data.sort_values(by=['time'], ascending=True, inplace=True)
input_data.reset_index(inplace=True)

In [11]:
input_data.head()

,unique_id,id,customer_id,load_amount,time
0,15887_528,15887,528,$3318.47,2000-01-01T00:00:00Z
1,30081_154,30081,154,$1413.18,2000-01-01T01:01:22Z
2,26540_426,26540,426,$404.56,2000-01-01T02:02:44Z
3,10694_1,10694,1,$785.11,2000-01-01T03:04:06Z
4,15089_205,15089,205,$2247.28,2000-01-01T04:05:28Z


In [12]:
input_data[input_data['unique_id']=='6928_562']

,unique_id,id,customer_id,load_amount,time
108,6928_562,6928,562,$5255.16,2000-01-05T14:27:36Z


* **How many users?**  50
* **How many transactions per user per day? per week? per month?**
* **What is the average transaction amount per user? per day? per week?**

In [13]:
# input_data['load_amount']= input_data.load_amount.replace('[\$,]', '', regex=True).astype(float)


In [14]:
# input_data['load_amount'].mean()

In [15]:
# plt.hist(input_data.load_amount, bins = 50)

In [16]:
# plt.scatter(input_data[input_data['customer_id']==1].time, input_data[input_data['customer_id']==1].customer_id)

In [17]:
# input_data[input_data['customer_id']==1].count(), input_data[input_data['customer_id']==1].min(), input_data[input_data['customer_id']==1].max(), input_data[input_data['customer_id']==1].mean()

In [18]:
customer1_df = input_data[input_data['customer_id']==1]
customer1_df

,unique_id,id,customer_id,load_amount,time
3,10694_1,10694,1,$785.11,2000-01-01T03:04:06Z
18,18705_1,18705,1,$3628.88,2000-01-01T18:24:36Z
66,11456_1,11456,1,$2566.29,2000-01-03T19:30:12Z
165,11114_1,11114,1,$2887.30,2000-01-08T00:45:30Z
171,7485_1,7485,1,$1920.88,2000-01-08T06:53:42Z
223,16907_1,16907,1,$4972.22,2000-01-10T12:04:46Z
229,31045_1,31045,1,$5992.27,2000-01-10T18:12:58Z
276,7806_1,7806,1,$4421.55,2000-01-12T18:17:12Z
496,10262_1,10262,1,$962.12,2000-01-22T03:17:52Z
498,19749_1,19749,1,$4770.96,2000-01-22T05:20:36Z


### Exhaustive Rules


In [21]:
cd ..

/content/velocity_limits


In [22]:
ls

rules.csv  takehome/  Velocity_limits.ipynb


In [23]:
rules_df = pd.read_csv("rules.csv")
rules_df

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
0,1,0,d,d,d,d,Decline - load ID not unique 1
1,2,0,d,d,d,d,Decline - load ID not unique 2
2,3,0,d,d,d,d,Decline - load ID not unique 3
3,4,0,d,d,d,d,Decline - load ID not unique 4
4,5,0,d,d,d,d,Decline - load ID not unique 5
5,6,0,d,d,d,d,Decline - load ID not unique 6
6,7,0,d,d,d,d,Decline - load ID not unique 7
7,8,0,d,d,d,d,Decline - load ID not unique 8
8,9,0,d,d,d,d,Decline - load ID not unique 9
9,10,0,d,d,d,d,Decline - load ID not unique 10


In [24]:

class Customer: 
  def __init__(self, customer_id, loads=None, daily_deposit_limit=5000, weekly_deposit_limit=20000, daily_loads_limit=3, initial_bal=0, 
               current_daily_deposits=0, current_weekly_deposits=0, current_daily_loads= 0 ):

    self.customer_id = customer_id
    self.loads = []
    self.daily_deposit_limit =  daily_deposit_limit
    self.weekly_deposit_limit =weekly_deposit_limit 
    self.daily_loads_limit = daily_loads_limit 
    self.balance = initial_bal

    self.current_daily_deposits =current_daily_deposits
    self.current_weekly_deposits = current_daily_deposits
    self.current_daily_loads = current_daily_loads


    self.daily_deposits_left = daily_deposit_limit
    self.weekly_deposits_left = weekly_deposit_limit 
    self.daily_loads_left = daily_loads_limit

    self.load_hist = pd.DataFrame({'customer_id':self.customer_id, 
                                   'load_id': [], 
                                   'load_amt': [], 
                                   'load_time': [], 
                                   'load_week_num':[],
                                   'load_day_num':[],
                                   'approved_status':[], 
                                   'ruleA':[],
                                   'ruleB':[],
                                   'rule1':[], 
                                   'rule2':[], 
                                   'rule3':[]})



  def get_daily_deposits_left(self): 
    if hasattr(self, 'valid_loads') and len(self.valid_loads)>0:
      self.daily_deposits_left = self.valid_loads[self.valid_loads.load_day_num ==self.valid_loads.iloc[-1].load_day_num].daily_deps_left.iloc[-1]

  def get_weekly_deposits_left(self): 
    if hasattr(self, 'valid_loads') and len(self.valid_loads)>0:
      self.weekly_deposits_left = self.valid_loads[self.valid_loads.load_week_num ==self.valid_loads.iloc[-1].load_week_num].weekly_deps_left.iloc[-1]

  def get_daily_loads_left(self): 
    if hasattr(self, 'valid_loads') and len(self.valid_loads)>0:
      self.daily_loads_left = self.valid_loads[self.valid_loads['load_day_num']==self.loads[-1].load_day_num]['num_daily_deps_left'].iloc[-1]


  def update_load_history(self): 
    
    self.load_hist = self.load_hist.append({'customer_id': self.customer_id, \
                                            'load_id': self.loads[-1].load_id, \
                                            'load_amt':self.loads[-1].load_amt, \
                                            'load_time': self.loads[-1].load_time, \
                                            'load_week_num':self.loads[-1].load_week_num,\
                                            'load_day_num':self.loads[-1].load_day_num,\
                                            'approved_status': self.ruleA&self.ruleB&self.rule1&self.rule2&self.rule3, 
                                            'ruleA': self.ruleA, 
                                            'ruleB': self.ruleB,
                                            'rule1': self.rule1, 
                                            'rule2': self.rule2, 
                                            'rule3': self.rule3
                                            }, ignore_index=True).copy(deep=True)

    self.load_hist['load_week_num'] = self.load_hist['load_week_num'].astype(int)
    self.load_hist['load_day_num'] = self.load_hist['load_day_num'].astype(int) 

    self.load_hist['approved_status'] = self.load_hist['approved_status'].astype(bool)  
    self.load_hist['ruleA'] = self.load_hist['ruleA'].astype(bool) 
    self.load_hist['ruleB'] = self.load_hist['ruleB'].astype(bool) 
    self.load_hist['rule1'] = self.load_hist['rule1'].astype(bool) 
    self.load_hist['rule2'] = self.load_hist['rule2'].astype(bool) 
    self.load_hist['rule3'] = self.load_hist['rule3'].astype(bool) 


    

  def process_valid_loads(self):
    # keeping only the valid loads (no dupes allowed)
    self.valid_loads = self.load_hist[self.load_hist['approved_status']==True].copy(deep=True)

    df1 = self.valid_loads.set_index('load_time').sort_index().copy(deep=True)

    df2 = pd.DataFrame(df1.groupby('load_week_num')['load_amt'].transform(pd.Series.cumsum)).copy(deep=True)
    df3 = pd.DataFrame(df1.groupby('load_day_num')['load_amt'].transform(pd.Series.cumsum)).copy(deep=True)
    df4 = pd.DataFrame(df1.groupby(['load_day_num']).cumcount()+1).copy(deep=True)
    df4.columns=['num_daily_deps']


    self.valid_loads['cum_weekly_deps'] = list(pd.merge(self.valid_loads, df2, how='left', left_on =['load_time'], right_on=['load_time'])['load_amt_y'])
    self.valid_loads['cum_daily_deps'] = list(pd.merge(self.valid_loads, df3, how='left', left_on =['load_time'], right_on=['load_time'])['load_amt_y'])
    self.valid_loads['num_daily_deps'] = list(pd.merge(self.valid_loads, df4, how='left', left_on =['load_time'], right_on=['load_time'])['num_daily_deps'])


    self.valid_loads['weekly_deps_left'] = self.weekly_deposit_limit - self.valid_loads['cum_weekly_deps']
    self.valid_loads['daily_deps_left'] = self.daily_deposit_limit - self.valid_loads['cum_daily_deps']
    self.valid_loads['num_daily_deps_left'] = self.daily_loads_limit - self.valid_loads['num_daily_deps']




  def process_load(self, load):
    self.balance += float(load.load_amt)
    self.loads.append(load)

    if hasattr(self, 'valid_loads') and len(self.valid_loads)>0:
      if load.load_day_num not in list(self.valid_loads.load_day_num): 
        self.daily_deposits_left = self.daily_deposit_limit
        self.daily_loads_left = self.daily_loads_limit

      if load.load_week_num not in list(self.valid_loads.load_week_num): 
        #print ("load.load_week_num not in list(self.valid_loads.load_week_num): is true")
        self.weekly_deposits_left = self.weekly_deposit_limit


    

    #3 rules: 
    # 1) A maximum of  $5,000  can be loaded per day
    # 2) A maximum of  $20,000  can be loaded per week
    # 3) A maximum of 3 loads can be performed per day, regardless of amount


    self.ruleA = load.load_id not in list(self.load_hist.load_id)
    self.ruleB = load.load_time not in list(self.load_hist.load_time) 
    self.rule1 = self.daily_deposits_left > float(load.load_amt)
    self.rule2 = self.daily_loads_left >= 1 
    self.rule3 = self.weekly_deposits_left > float(load.load_amt) 

    self.rule_status = str(self.ruleA) + " " + str(self.ruleB) + " " + str(self.rule1) +  " " + str(self.rule2) + " " + str(self.rule3)

    if not self.ruleA:
      print("ERROR: Load ID already exists")

    if not self.ruleB: 
      print("ERROR: Time stamp already exists for same customer")

    if not self.rule1: 
      print("ERROR: No room for daily deposits left")

    if not self.rule2: 
      print("ERROR: Number of allowed daily deposits exceeded")

    if not self.rule3: 
      print("ERROR: No room for weekly deposits left")


    self.update_load_history()
    self.process_valid_loads()


    if self.ruleA & self.ruleB & self.rule1 & self.rule2 & self.rule3 : 
      self.current_daily_deposits+=float(load.load_amt)
      self.current_weekly_deposits+=float(load.load_amt)
      self.current_daily_loads+=1

      self.get_daily_deposits_left()

      self.get_weekly_deposits_left()


      self.get_daily_loads_left()

      self.loads[-1].approved_status = True 
      print("All Rules Passed ... Load Added")

    else: 
      print("Load Rejected")

    return None

  def print_status(self): 
    print("Customer_id:{}".format(self.customer_id))
    print("Current Deposit Balance:{}".format(self.balance))
    print("Current Daily Deposits:{}".format(self.current_daily_deposits))
    print("Current Weekly Deposits:{}".format(self.current_weekly_deposits))
    print("Current Daily Loads:{}".format(self.current_daily_loads))

    print("Daily Deposits Left:{}".format(self.daily_deposits_left))
    print("Weekly Deposits Left:{}".format(self.weekly_deposits_left))
    print("Daily Loads Left:{}".format(self.daily_loads_left))



In [25]:

# import datetime as datetime 
class Load: 
  def __init__(self, load_id, load_amt, load_time):
    self.load_id = load_id
    self.load_amt = float(load_amt[1:])
    self.load_time = load_time 
    self.approved_status = False

    self.datetime_object = datetime.strptime(self.load_time, "%Y-%m-%dT%H:%M:%SZ")
    self.load_week_num = self.datetime_object.year + self.datetime_object.isocalendar()[1]

    import datetime as dt

    self.load_day_num = self.datetime_object.year + (dt.date(self.datetime_object.year, self.datetime_object.month, self.datetime_object.day) - dt.date(self.datetime_object.year,1,1)).days + 1

    # self.day_num = (datetime.date(self.datetime_object.year, self.datetime_object.month, self.datetime_object.day) - datetime.date(year,1,1)).days + 1



    

Daily Dep Amount Exceeded

In [29]:
rules_df.iloc[24:28]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
24,25,1,1,0,d,d,Decline - Daily Deps left limit exceeded 1
25,26,1,1,0,d,d,Decline - Daily Deps left limit exceeded 2
26,27,1,1,0,d,d,Decline - Daily Deps left limit exceeded 3
27,28,1,1,0,d,d,Decline - Daily Deps left limit exceeded 4


In [30]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$7785.11', '2000-01-01T03:04:06Z'))

ERROR: No room for daily deposits left
Load Rejected


In [31]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,7785.11,2000-01-01T03:04:06Z,2052,2001,False,True,True,False,True,True


In [32]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left


In [33]:
cust1.rule_status

'True True False True True'

### Approval Case

In [38]:
rules_df.iloc[[31]]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
31,32,1,1,1,1,1,Approve


In [39]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$3785.11', '2000-01-01T03:04:06Z'))

All Rules Passed ... Load Added


In [40]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,3785.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [41]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,3785.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,3785.11,3785.11,1,16214.89,1214.89,2


### Tests 1 - 15: Load ID is not unique

In [42]:
rules_df.iloc[0:15]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
0,1,0,d,d,d,d,Decline - load ID not unique 1
1,2,0,d,d,d,d,Decline - load ID not unique 2
2,3,0,d,d,d,d,Decline - load ID not unique 3
3,4,0,d,d,d,d,Decline - load ID not unique 4
4,5,0,d,d,d,d,Decline - load ID not unique 5
5,6,0,d,d,d,d,Decline - load ID not unique 6
6,7,0,d,d,d,d,Decline - load ID not unique 7
7,8,0,d,d,d,d,Decline - load ID not unique 8
8,9,0,d,d,d,d,Decline - load ID not unique 9
9,10,0,d,d,d,d,Decline - load ID not unique 10


In [43]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$385.11', '2000-01-01T03:04:06Z'))

All Rules Passed ... Load Added


In [44]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,385.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [45]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,385.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,385.11,385.11,1,19614.89,4614.89,2


In [46]:
cust1.process_load(Load('10694', '$385.11', '2000-01-01T03:05:06Z'))

ERROR: Load ID already exists
Load Rejected


### Time stamp is not unique

In [47]:
rules_df.iloc[16:23]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
16,17,1,0,0,0,0,Decline - time stamp not unique 1
17,18,1,0,0,0,1,Decline - time stamp not unique 2
18,19,1,0,0,1,0,Decline - time stamp not unique 3
19,20,1,0,0,1,1,Decline - time stamp not unique 4
20,21,1,0,1,0,0,Decline - time stamp not unique 5
21,22,1,0,1,0,1,Decline - time stamp not unique 6
22,23,1,0,1,1,0,Decline - time stamp not unique 7


In [48]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$385.11', '2000-01-01T03:04:06Z'))

All Rules Passed ... Load Added


In [49]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,385.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [50]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,385.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,385.11,385.11,1,19614.89,4614.89,2


In [51]:
cust1.process_load(Load('10695', '$3.11', '2000-01-01T03:04:06Z'))

ERROR: Time stamp already exists for same customer
Load Rejected


### Daily Deposit Amount Is Exceeded

In [52]:
rules_df.iloc[24:28]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
24,25,1,1,0,d,d,Decline - Daily Deps left limit exceeded 1
25,26,1,1,0,d,d,Decline - Daily Deps left limit exceeded 2
26,27,1,1,0,d,d,Decline - Daily Deps left limit exceeded 3
27,28,1,1,0,d,d,Decline - Daily Deps left limit exceeded 4


In [53]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$6000.11', '2000-01-01T03:04:06Z'))

ERROR: No room for daily deposits left
Load Rejected


In [54]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,6000.11,2000-01-01T03:04:06Z,2052,2001,False,True,True,False,True,True


In [55]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left


### Number of daily loads is exceeded - same day

In [56]:
rules_df.iloc[28:30]

,Rule Number,Load ID is unique,time stamp is unique,daily_deps_left > load.amt,daily_loads_left >= 0,weekly_deps_left > load.amt,System Response
28,29,1,1,1,0,d,Decline - Daily loads left limit exceeded 1
29,30,1,1,1,0,d,Decline - Daily loads left limit exceeded 2


In [57]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$600.11', '2000-01-01T03:04:06Z'))

All Rules Passed ... Load Added


In [58]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [59]:
cust1.process_load(Load('10695', '$600.11', '2000-01-01T03:05:06Z'))

All Rules Passed ... Load Added


In [60]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True


In [61]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True,1200.22,1200.22,2,18799.78,3799.78,1


In [62]:
cust1.process_load(Load('10696', '$600.11', '2000-01-01T03:06:06Z'))

All Rules Passed ... Load Added


In [63]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True
2,1,10696,600.11,2000-01-01T03:06:06Z,2052,2001,True,True,True,True,True,True


In [64]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True,1200.22,1200.22,2,18799.78,3799.78,1
2,1,10696,600.11,2000-01-01T03:06:06Z,2052,2001,True,True,True,True,True,True,1800.33,1800.33,3,18199.67,3199.67,0


In [65]:
cust1.process_load(Load('10697', '$600.11', '2000-01-01T03:09:06Z'))

ERROR: Number of allowed daily deposits exceeded
Load Rejected


### Number of Daily loads is exceeded - multiple days 

In [66]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$600.11', '2000-01-01T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [67]:
cust1.process_load(Load('10695', '$600.11', '2000-01-01T03:05:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True


In [68]:
cust1.process_load(Load('10696', '$600.11', '2000-01-01T03:06:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True
2,1,10696,600.11,2000-01-01T03:06:06Z,2052,2001,True,True,True,True,True,True


In [69]:
cust1.process_load(Load('10702', '$600.11', '2000-01-01T03:11:06Z'))
cust1.load_hist

ERROR: Number of allowed daily deposits exceeded
Load Rejected


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True
2,1,10696,600.11,2000-01-01T03:06:06Z,2052,2001,True,True,True,True,True,True
3,1,10702,600.11,2000-01-01T03:11:06Z,2052,2001,False,True,True,True,False,True


In [70]:
cust1.process_load(Load('10704', '$600.11', '2000-01-02T03:12:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-01T03:05:06Z,2052,2001,True,True,True,True,True,True
2,1,10696,600.11,2000-01-01T03:06:06Z,2052,2001,True,True,True,True,True,True
3,1,10702,600.11,2000-01-01T03:11:06Z,2052,2001,False,True,True,True,False,True
4,1,10704,600.11,2000-01-02T03:12:06Z,2052,2002,True,True,True,True,True,True


#### Daily loads over multiple days 

In [73]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$600.11', '2000-01-01T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [74]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2


In [75]:
cust1.process_load(Load('10695', '$600.11', '2000-01-02T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True


In [76]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,1200.22,600.11,1,18799.78,4399.89,2


In [77]:
cust1.process_load(Load('10696', '$600.11', '2000-01-02T03:05:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,600.11,2000-01-02T03:05:06Z,2052,2002,True,True,True,True,True,True


In [78]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,1200.22,600.11,1,18799.78,4399.89,2
2,1,10696,600.11,2000-01-02T03:05:06Z,2052,2002,True,True,True,True,True,True,1800.33,1200.22,2,18199.67,3799.78,1


In [79]:
cust1.process_load(Load('10697', '$600.11', '2000-01-02T03:06:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,600.11,2000-01-02T03:05:06Z,2052,2002,True,True,True,True,True,True
3,1,10697,600.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True


In [80]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,600.11,600.11,1,19399.89,4399.89,2
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,1200.22,600.11,1,18799.78,4399.89,2
2,1,10696,600.11,2000-01-02T03:05:06Z,2052,2002,True,True,True,True,True,True,1800.33,1200.22,2,18199.67,3799.78,1
3,1,10697,600.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True,2400.44,1800.33,3,17599.56,3199.67,0


In [81]:
cust1.process_load(Load('10698', '$600.11', '2000-01-02T03:07:06Z'))
cust1.load_hist

ERROR: Number of allowed daily deposits exceeded
Load Rejected


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,600.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,600.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,600.11,2000-01-02T03:05:06Z,2052,2002,True,True,True,True,True,True
3,1,10697,600.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True
4,1,10698,600.11,2000-01-02T03:07:06Z,2052,2002,False,True,True,True,False,True


### Daily Deposit Amount over multiple days 

In [82]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$300.11', '2000-01-01T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [83]:
cust1.process_load(Load('10695', '$3000.11', '2000-01-02T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True


In [84]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,3300.22,3000.11,1,16699.78,1999.89,2


In [85]:
cust1.process_load(Load('10696', '$3000.11', '2000-01-02T03:05:06Z'))
cust1.load_hist

ERROR: No room for daily deposits left
Load Rejected


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,3000.11,2000-01-02T03:05:06Z,2052,2002,False,True,True,False,True,True


In [86]:
cust1.process_load(Load('10697', '$300.11', '2000-01-02T03:06:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,3000.11,2000-01-02T03:05:06Z,2052,2002,False,True,True,False,True,True
3,1,10697,300.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True


In [87]:
cust1.process_load(Load('10698', '$300.11', '2000-01-04T03:05:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,3000.11,2000-01-02T03:05:06Z,2052,2002,False,True,True,False,True,True
3,1,10697,300.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True
4,1,10698,300.11,2000-01-04T03:05:06Z,2001,2004,True,True,True,True,True,True


In [88]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2
1,1,10695,3000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,3300.22,3000.11,1,16699.78,1999.89,2
3,1,10697,300.11,2000-01-02T03:06:06Z,2052,2002,True,True,True,True,True,True,3600.33,3300.22,2,16399.67,1699.78,1
4,1,10698,300.11,2000-01-04T03:05:06Z,2001,2004,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2


### Weekly deposit amount over same week

In [89]:
cust1 = Customer(1)
cust1.process_load(Load('10694', '$300.11', '2000-01-01T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True


In [90]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2


In [91]:
cust1.process_load(Load('10695', '$4000.11', '2000-01-02T03:04:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True


In [92]:
cust1.process_load(Load('10696', '$4000.11', '2000-01-03T03:05:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True


In [93]:
cust1.process_load(Load('10698', '$4000.11', '2000-01-05T03:06:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True


In [94]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,4300.22,4000.11,1,15699.78,999.89,2
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True,4000.11,4000.11,1,15999.89,999.89,2
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True,8000.22,4000.11,1,11999.78,999.89,2


In [95]:
cust1.process_load(Load('10700', '$4999.00', '2000-01-06T03:07:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True


In [96]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,4300.22,4000.11,1,15699.78,999.89,2
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True,4000.11,4000.11,1,15999.89,999.89,2
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True,8000.22,4000.11,1,11999.78,999.89,2
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True,12999.22,4999.00,1,7000.78,1.00,2


In [97]:
cust1.process_load(Load('10701', '$4999.00', '2000-01-07T03:07:06Z'))
cust1.load_hist

All Rules Passed ... Load Added


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True
5,1,10701,4999.00,2000-01-07T03:07:06Z,2001,2007,True,True,True,True,True,True


In [98]:
cust1.valid_loads

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3,cum_weekly_deps,cum_daily_deps,num_daily_deps,weekly_deps_left,daily_deps_left,num_daily_deps_left
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True,300.11,300.11,1,19699.89,4699.89,2
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True,4300.22,4000.11,1,15699.78,999.89,2
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True,4000.11,4000.11,1,15999.89,999.89,2
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True,8000.22,4000.11,1,11999.78,999.89,2
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True,12999.22,4999.00,1,7000.78,1.00,2
5,1,10701,4999.00,2000-01-07T03:07:06Z,2001,2007,True,True,True,True,True,True,17998.22,4999.00,1,2001.78,1.00,2


In [99]:
cust1.process_load(Load('10702', '$4999.00', '2000-01-08T03:07:06Z'))
cust1.load_hist

ERROR: No room for weekly deposits left
Load Rejected


,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True
5,1,10701,4999.00,2000-01-07T03:07:06Z,2001,2007,True,True,True,True,True,True
6,1,10702,4999.00,2000-01-08T03:07:06Z,2001,2008,False,True,True,True,True,False


In [100]:
cust1.load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,1,10694,300.11,2000-01-01T03:04:06Z,2052,2001,True,True,True,True,True,True
1,1,10695,4000.11,2000-01-02T03:04:06Z,2052,2002,True,True,True,True,True,True
2,1,10696,4000.11,2000-01-03T03:05:06Z,2001,2003,True,True,True,True,True,True
3,1,10698,4000.11,2000-01-05T03:06:06Z,2001,2005,True,True,True,True,True,True
4,1,10700,4999.00,2000-01-06T03:07:06Z,2001,2006,True,True,True,True,True,True
5,1,10701,4999.00,2000-01-07T03:07:06Z,2001,2007,True,True,True,True,True,True
6,1,10702,4999.00,2000-01-08T03:07:06Z,2001,2008,False,True,True,True,True,False


In [101]:
cust1.load_hist.load_id.iloc[0]

'10694'

In [102]:
class Controller: 
  def __init__(self): 
    self.customers = {}

In [103]:
input_payloads_df = input_data[input_data.columns[1:]].copy(deep=True)
input_payloads_df.head()

,id,customer_id,load_amount,time
0,15887,528,$3318.47,2000-01-01T00:00:00Z
1,30081,154,$1413.18,2000-01-01T01:01:22Z
2,26540,426,$404.56,2000-01-01T02:02:44Z
3,10694,1,$785.11,2000-01-01T03:04:06Z
4,15089,205,$2247.28,2000-01-01T04:05:28Z


In [104]:
input_payloads_list= []
for i in range(0, len(input_payloads_df)): 
  input_payloads_list.append({'id':input_payloads_df.iloc[i].id, 
  'customer_id':input_payloads_df.iloc[i].customer_id, 
  'load_amount':input_data.iloc[i].load_amount, 
  'time': input_data.iloc[i].time})

In [114]:
controller = Controller()

output_txt = ""

for payload in input_payloads_list: 
  print("input:{}".format(payload))
  if payload['customer_id'] not in controller.customers: 
    controller.customers[payload['customer_id']] = Customer(payload['customer_id'])

  controller.customers[payload['customer_id']].process_load(Load(payload['id'], payload['load_amount'], payload['time']))
  accepted_status = controller.customers[payload['customer_id']].load_hist.approved_status.iloc[-1] 
  customer_id = controller.customers[payload['customer_id']].customer_id
  id =  controller.customers[payload['customer_id']].load_hist.load_id.iloc[-1]

  output = {"accepted": accepted_status, "customer_id": customer_id, "id": id}
  output_txt += str(output)
  print("output:{}".format(output))

input:{'id': 15887, 'customer_id': 528, 'load_amount': '$3318.47', 'time': '2000-01-01T00:00:00Z'}
All Rules Passed ... Load Added
output:{'accepted': True, 'customer_id': 528, 'id': 15887.0}
input:{'id': 30081, 'customer_id': 154, 'load_amount': '$1413.18', 'time': '2000-01-01T01:01:22Z'}
All Rules Passed ... Load Added
output:{'accepted': True, 'customer_id': 154, 'id': 30081.0}
input:{'id': 26540, 'customer_id': 426, 'load_amount': '$404.56', 'time': '2000-01-01T02:02:44Z'}
All Rules Passed ... Load Added
output:{'accepted': True, 'customer_id': 426, 'id': 26540.0}
input:{'id': 10694, 'customer_id': 1, 'load_amount': '$785.11', 'time': '2000-01-01T03:04:06Z'}
All Rules Passed ... Load Added
output:{'accepted': True, 'customer_id': 1, 'id': 10694.0}
input:{'id': 15089, 'customer_id': 205, 'load_amount': '$2247.28', 'time': '2000-01-01T04:05:28Z'}
All Rules Passed ... Load Added
output:{'accepted': True, 'customer_id': 205, 'id': 15089.0}
input:{'id': 3211, 'customer_id': 409, 'load_a

In [118]:
controller.customers[545].load_hist

,customer_id,load_id,load_amt,load_time,load_week_num,load_day_num,approved_status,ruleA,ruleB,rule1,rule2,rule3
0,545,25179.0,5338.75,2000-01-08T13:01:54Z,2001,2008,False,True,True,False,True,True
1,545,15495.0,5308.56,2000-01-12T01:55:20Z,2002,2012,False,True,True,False,True,True
2,545,15605.0,5631.18,2000-01-14T17:20:04Z,2002,2014,False,True,True,False,True,True
3,545,10083.0,3580.90,2000-01-16T13:18:50Z,2002,2016,True,True,True,True,True,True
4,545,5335.0,5518.54,2000-01-17T04:39:20Z,2003,2017,False,True,True,False,True,True
5,545,29910.0,279.17,2000-01-19T03:42:12Z,2003,2019,True,True,True,True,True,True
6,545,28249.0,3265.54,2000-01-20T18:34:08Z,2003,2020,True,True,True,True,True,True
7,545,3111.0,4007.33,2000-01-22T13:31:32Z,2003,2022,True,True,True,True,True,True
8,545,19377.0,4404.66,2000-01-22T21:42:28Z,2003,2022,False,True,True,False,True,True
9,545,29415.0,3047.54,2000-01-23T11:00:14Z,2003,2023,True,True,True,True,True,True


In [111]:
print(output_txt)

{'accepted': True, 'customer_id': 528, 'id': 15887.0}{'accepted': True, 'customer_id': 154, 'id': 30081.0}{'accepted': True, 'customer_id': 426, 'id': 26540.0}{'accepted': True, 'customer_id': 1, 'id': 10694.0}{'accepted': True, 'customer_id': 205, 'id': 15089.0}{'accepted': True, 'customer_id': 409, 'id': 3211.0}{'accepted': True, 'customer_id': 630, 'id': 27106.0}{'accepted': False, 'customer_id': 273, 'id': 7528.0}{'accepted': True, 'customer_id': 800, 'id': 27947.0}{'accepted': True, 'customer_id': 647, 'id': 20790.0}{'accepted': True, 'customer_id': 698, 'id': 12408.0}{'accepted': False, 'customer_id': 528, 'id': 11429.0}{'accepted': False, 'customer_id': 630, 'id': 16631.0}{'accepted': True, 'customer_id': 443, 'id': 22413.0}{'accepted': True, 'customer_id': 749, 'id': 10563.0}{'accepted': False, 'customer_id': 800, 'id': 26078.0}{'accepted': True, 'customer_id': 154, 'id': 11353.0}{'accepted': True, 'customer_id': 358, 'id': 19189.0}{'accepted': True, 'customer_id': 1, 'id': 187

### Creating Output Text File 

In [112]:
output_text_file = open("output.txt", "w")
output_text_file.write(output_txt)
output_text_file.close()

In [113]:
ls

output.txt  rules.csv  takehome/  Velocity_limits.ipynb
